In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import tensorflow as tf


train_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/ml21_final/task1_2021.training.csv')
test_df = pd.read_csv('./drive/MyDrive/Colab Notebooks/ml21_final/task1_2021.test.csv')
train_y = train_df['target']
train_x = train_df.drop(['target'], axis=1)

scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_df)

mse = make_scorer(mean_squared_error, squared=False)
print(tf.test.is_gpu_available())

In [ ]:

class DNN(tf.keras.Model):
  def __init__(self):
    super(DNN,self).__init__()
    self.dense1 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense3 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense4 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense5 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense6 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.dense7 = tf.keras.layers.Dense(30, activation=tf.nn.relu)
    self.output_layer = tf.keras.layers.Dense(1)

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    x = self.dense3(x)
    x = self.dense4(x)
    x = self.dense5(x)
    x = self.dense6(x)
    x = self.dense7(x)
    return self.output_layer(x)

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    elif epoch < 90:
        return 1e-3
    elif epoch < 400:
        return 1e-5
    elif epoch < 900:
        return 1e-6
    elif epoch < 10000:
        return 1e-7


callback_early = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=100,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)

callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

batch_size = 100
epochs = 10000
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)
model = DNN()
model.compile(loss="mean_squared_error", optimizer=opt, metrics=["mean_squared_error"])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback_lr,callback_early], verbose=1)
model.summary()

In [ ]:
pred = model.predict(train_x).reshape(-1)
mean_squared_error(train_y, pred, squared=False)

In [ ]:
from pandas import RangeIndex

nn_pred = model.predict(test_x)
output = nn_pred.reshape(-1)

pd.Series(output, index=RangeIndex(1, 2001), name='Predicted').to_csv('ans_nn_3.csv',index_label='id')